In [847]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

### Account cleanen

In [848]:
df_account = pd.read_csv('../data_clean/Account_fixed.csv')

# alleen de actieve accounts
df_account = df_account[df_account['account_reden_van_status'] != 'Inactief']
df_account = df_account[df_account['account_reden_van_status'] != 'Stopzetting']

# onnodige kolommen verwijderen
df_account.drop(['account_industriezone_naam_',
           'account_oprichtingsdatum',
           'account_reden_van_status',
           'account_status',
           'account_voka_nr_',
           'account_is_voka_entiteit',
], axis=1, inplace=True)

# alleen de accounts uit Oost-Vlaanderen
df_account = df_account[df_account['account_adres_provincie'] == 'Oost-Vlaanderen']

# account adres samenvoegen
plaats = df_account['account_adres_plaats'].str.lower().str.capitalize()
postcode = df_account['account_adres_postcode'].astype(str)
subregio = df_account['account_adres_geografische_subregio'].str.lower().str.capitalize()
provincie = df_account['account_adres_provincie'].str.lower().str.capitalize()
land = df_account['account_adres_land'].str.lower().str.capitalize()

df_account['account_adres'] = plaats + ' ' + postcode + ' ' + subregio + ' ' + provincie + ' ' + land

df_account.drop(['account_adres_geografische_regio', 
           'account_adres_geografische_subregio', 
           'account_adres_plaats', 'account_adres_postcode', 
           'account_adres_provincie', 'account_adres_land'], axis=1, inplace=True)

# account ondernemingstype samenvoegen
df_account['account_onderneming'] = df_account['account_ondernemingstype'] + ', ' \
                            + df_account['account_ondernemingsaard'] + ', ' \
                            + df_account['account_primaire_activiteit'] + ' ' \

df_account.drop(['account_ondernemingstype', 
           'account_ondernemingsaard', 
           'account_primaire_activiteit'], axis=1, inplace=True)

# account ondernemingstype strings opschonen
df_account['account_onderneming'] = df_account['account_onderneming'].str.replace('unknown', '')
df_account['account_onderneming'] = df_account['account_onderneming'].str.strip()
df_account['account_onderneming'] = df_account['account_onderneming'].str.lower().str.capitalize()

df_account.shape

(41859, 3)

### Contact cleanen

In [849]:
df_contact = pd.read_csv('../data_clean/Contact_fixed.csv')

df_contact = df_contact[df_contact['contact_status'] != 'Inactief']
df_contact.drop(['contact_status', 'contact_voka_medewerker'], axis=1, inplace=True)

df_contact.shape

(393514, 4)

### Mergen van Account en Contact zodat alleen de contacten van Oost-Vlaanderen overblijven

Inner join, anders 11000 accounts zonder contactpersoon

In [850]:
df_account_contact = df_account.merge(df_contact, left_on='account_account_id', right_on='contact_account', how='inner')
df_account_contact.drop(['contact_account'], axis=1, inplace=True)
df_account_contact.shape

(67299, 6)

##### Persoon cleanen

In [851]:
# df_persoon = pd.read_csv('../data_clean/Persoon_fixed.csv')

# df_persoon = df_persoon[df_persoon['persoon_mail_regio_oost_vlaanderen'] == 1]
# df_persoon = df_persoon[df_persoon['persoon_reden_van_status'] == 'Actief']

# for col in df_persoon.columns:
#     if col.__contains__('persoon_mail_regio'):
#         df_persoon.drop(col, axis=1, inplace=True)

# df_persoon.drop(['persoon_persoonsnr_', 'persoon_web_login', 'persoon_reden_van_status'], axis=1, inplace=True)

# df_persoon.shape

### Account_contact mergen met persoon

In [852]:
# df_account_contact_persoon = df_account_contact.merge(df_persoon, left_on='contact_persoon_id', right_on='persoon_persoon_id', how='left')
# df_account_contact_persoon.drop(['contact_persoon_id'], axis=1, inplace=True)
# df_account_contact_persoon.shape

In [853]:
# df_account_contact_persoon.info()

##### Persoon niet meer gebruiken -> maar 4374 non-null

In [854]:
df_account_contact.drop(['contact_persoon_id'], axis=1, inplace=True)

### Account_contact mergen met contactfunctie en functie

In [855]:
df_contact_functie = pd.read_csv('../data_clean/Contact_functie_fixed.csv')
df_contact_functie.columns

Index(['contactfunctie_contactpersoon', 'contactfunctie_functie'], dtype='object')

In [856]:
df_functie = pd.read_csv('../data_clean/Functie_fixed.csv')
df_functie.columns

Index(['functie_functie_id', 'functie_naam'], dtype='object')

Merge contact functie

In [857]:
df_account_contact = df_account_contact.merge(df_contact_functie, left_on='contact_contactpersoon_id', right_on='contactfunctie_contactpersoon', how='inner')
df_account_contact.drop(['contactfunctie_contactpersoon'], axis=1, inplace=True)
df_account_contact.shape

(90938, 6)

Merge met functie

In [858]:
df_account_contact = df_account_contact.merge(df_functie, left_on='contactfunctie_functie', right_on='functie_functie_id', how='inner')
df_account_contact.drop(['contactfunctie_functie', 'functie_functie_id'], axis=1, inplace=True)
df_account_contact.shape

(90938, 6)

##### Functie naam kan verschillend zijn voor hetzelfde contact, dus die mergen we

In [859]:
# Define a custom aggregation function to merge 'functie_naam'
def merge_functie_naam(series):
    return ', '.join(series)

# Group by 'account_account_id' and 'contact_contactpersoon_id' and merge 'functie_naam'
df_account_contact = df_account_contact.groupby(
    ['account_account_id', 'contact_contactpersoon_id'], 
    as_index=False).agg({'functie_naam': merge_functie_naam, 
                         **{col: 'first' for col in df_account_contact.columns if col != 'functie_naam'}})

df_account_contact.shape

(67225, 6)

### Account_contact mergen met account_activiteitscode en activiteitscode

In [860]:
df_account_activiteitscode = pd.read_csv('../data_clean/Account_activiteitscode_fixed.csv')
print(df_account_activiteitscode.shape)
df_account_activiteitscode.columns

(14673, 3)


Index(['account_activiteitscode_account',
       'account_activiteitscode_activiteitscode',
       'account_activiteitscode_inf_account_inf_activiteitscodeid'],
      dtype='object')

In [861]:
df_activiteitscode = pd.read_csv('../data_clean/Activiteitscode_fixed.csv')
print(df_activiteitscode.shape)
df_activiteitscode.columns

(40, 3)


Index(['activiteitscode_naam', 'activiteitscode_activiteitscode_id',
       'activiteitscode_status'],
      dtype='object')

account_activiteitscode mergen met activiteitscode

In [862]:
df_activiteit = df_account_activiteitscode.merge(df_activiteitscode, left_on='account_activiteitscode_activiteitscode', right_on='activiteitscode_activiteitscode_id', how='inner')
df_activiteit = df_activiteit[df_activiteit['activiteitscode_status'] == 'Actief']
df_activiteit.drop(['account_activiteitscode_activiteitscode', 'activiteitscode_activiteitscode_id', 'activiteitscode_status', 'account_activiteitscode_inf_account_inf_activiteitscodeid'], axis=1, inplace=True)
df_activiteit.shape

(14665, 2)

activiteitscode naam combineren waar account niet uniek is

In [863]:
# where account_activiteitscode_account is not unique, combine the activiteitscode_naam
def merge_activiteitscode_naam(series):
    return ', '.join(series)

df_activiteit = df_activiteit.groupby(
    ['account_activiteitscode_account'], 
    as_index=False).agg({'activiteitscode_naam': merge_activiteitscode_naam, 
                         **{col: 'first' for col in df_activiteit.columns if col != 'activiteitscode_naam'}})

print(df_activiteit.shape)

df_activiteit.head()

(13408, 2)


,activiteitscode_naam,account_activiteitscode_account
0,Overige industrie & diensten,00002DAC-0A69-E111-B43A-00505680000A
1,Consultancy,00068436-F919-E211-9DAA-005056B06EB4
2,Technologische industrie & diensten,0009A6E6-2369-E111-B43A-00505680000A
3,Voeding,000D39CF-BE68-E111-B43A-00505680000A
4,Overige industrie & diensten,0016CAE8-BD68-E111-B43A-00505680000A


mergen met account_contact

In [864]:
df_account_contact = df_account_contact.merge(df_activiteit, left_on='account_account_id', right_on='account_activiteitscode_account', how='left')
df_account_contact.drop(['account_activiteitscode_account'], axis=1, inplace=True)
df_account_contact.shape

(67225, 7)

#####  Maar 16160 vd 67225 activiteitscodes zijn non-null

In [865]:
df_account_contact.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67225 entries, 0 to 67224
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   functie_naam               67225 non-null  object
 1   account_account_id         67225 non-null  object
 2   account_adres              67225 non-null  object
 3   account_onderneming        67225 non-null  object
 4   contact_contactpersoon_id  67225 non-null  object
 5   contact_functietitel       67225 non-null  object
 6   activiteitscode_naam       16160 non-null  object
dtypes: object(7)
memory usage: 3.6+ MB


### Account financiele data mergen met account_contact

In [866]:
df_account_financieel = pd.read_csv('../data_clean/Account_financiële_data_fixed.csv', sep=',')
df_account_financieel.drop(['financieledata_gewijzigd_op', 'financieledata_fte', 'financieledata_aantal_maanden'], axis=1, inplace=True)
df_account_financieel['financieledata_toegevoegde_waarde'] = df_account_financieel['financieledata_toegevoegde_waarde'].astype(str)

print(df_account_financieel.shape)
df_account_financieel.columns

(2030180, 3)


Index(['financieledata_ondernemingid', 'financieledata_boekjaar',
       'financieledata_toegevoegde_waarde'],
      dtype='object')

In [867]:
df_account_financieel.head(10)

,financieledata_ondernemingid,financieledata_boekjaar,financieledata_toegevoegde_waarde
0,00002DAC-0A69-E111-B43A-00505680000A,2007,unknown
1,00002DAC-0A69-E111-B43A-00505680000A,2008,unknown
2,00002DAC-0A69-E111-B43A-00505680000A,2009,unknown
3,00002DAC-0A69-E111-B43A-00505680000A,2010,unknown
4,00002DAC-0A69-E111-B43A-00505680000A,2011,unknown
5,00002DAC-0A69-E111-B43A-00505680000A,2012,unknown
6,00002DAC-0A69-E111-B43A-00505680000A,2013,69164
7,00002DAC-0A69-E111-B43A-00505680000A,2014,46718
8,00002DAC-0A69-E111-B43A-00505680000A,2015,23551
9,00002DAC-0A69-E111-B43A-00505680000A,2016,65481


In [868]:
df_account_contact_finance = df_account_contact.merge(df_account_financieel, left_on='account_account_id', right_on='financieledata_ondernemingid', how='left')
df_account_contact_finance.drop(['financieledata_ondernemingid'], axis=1, inplace=True)
df_account_contact_finance.shape

(687078, 9)

In [869]:
df_account_contact_finance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687078 entries, 0 to 687077
Data columns (total 9 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   functie_naam                       687078 non-null  object 
 1   account_account_id                 687078 non-null  object 
 2   account_adres                      687078 non-null  object 
 3   account_onderneming                687078 non-null  object 
 4   contact_contactpersoon_id          687078 non-null  object 
 5   contact_functietitel               687078 non-null  object 
 6   activiteitscode_naam               237628 non-null  object 
 7   financieledata_boekjaar            670924 non-null  float64
 8   financieledata_toegevoegde_waarde  670924 non-null  object 
dtypes: float64(1), object(8)
memory usage: 47.2+ MB


In [870]:
# turn financieledata_toegevoegde_waarde into a numeric value
df_account_contact_finance['financieledata_toegevoegde_waarde'] = df_account_contact_finance['financieledata_toegevoegde_waarde'].str.replace(',', '.')
df_account_contact_finance['financieledata_toegevoegde_waarde'] = df_account_contact_finance['financieledata_toegevoegde_waarde'].str.replace('unknown', '0')
df_account_contact_finance['financieledata_toegevoegde_waarde'] = df_account_contact_finance['financieledata_toegevoegde_waarde'].astype(float)

##### De financieel toegevoegde waarde optellen per account en de boekjaren hun range gebruiken (vb 2007 - 2022 => 15 jaar)

In [871]:
# group by 'account_account_id' and 'contact_contactpersoon_id' and merge 'financieledata_toegevoegde_waarde' by sum and 'financieledata_boekjaar' by max - min
df_account_contact_finance = df_account_contact_finance.groupby(
    ['account_account_id', 'contact_contactpersoon_id'], 
    as_index=False).agg({'financieledata_toegevoegde_waarde': 'sum', 
                         'financieledata_boekjaar': lambda x: x.max() - x.min(), 
                         **{col: 'first' for col in df_account_contact_finance.columns if col not in ['financieledata_toegevoegde_waarde', 'financieledata_boekjaar']}})

df_account_contact_finance.rename(columns={'financieledata_boekjaar': 'financieledata_boekjaar_range'}, inplace=True)

In [872]:
df_account_contact_finance.shape

(67225, 9)

##### Alle niet meer benodigde dataframes verwijderen

In [873]:
try:
    del df_account
    del df_contact
    del df_contact_functie
    del df_functie
    del df_account_financieel
    del df_account_contact
except:
    print('already deleted or non-existent')

### Afspraken cleanen en mergen

In [874]:
df_afspraak_acc_gelinkt = pd.read_csv('../data_clean/Afspraak_account_gelinkt_cleaned_fixed.csv', sep=',')
print(df_afspraak_acc_gelinkt.shape)
print(list(df_afspraak_acc_gelinkt.columns))
df_afspraak_acc_gelinkt.drop(['afspraak_account_gelinkt_account'], axis=1, inplace=True) 

(2934, 7)
['afspraak_account_gelinkt_afspraak_id', 'afspraak_account_gelinkt_thema', 'afspraak_account_gelinkt_subthema', 'afspraak_account_gelinkt_onderwerp', 'afspraak_account_gelinkt_eindtijd', 'afspraak_account_gelinkt_account', 'afspraak_account_gelinkt_keyphrases']


In [875]:
df_afspraak_betreft_acc = pd.read_csv('../data_clean/Afspraak_betreft_account_cleaned_fixed.csv', sep=',')
print(df_afspraak_betreft_acc.shape)
print(list(df_afspraak_betreft_acc.columns))
df_afspraak_betreft_acc.drop(['afspraak_betreft_account_betreft_id'], axis=1, inplace=True)

(4876, 7)
['afspraak_betreft_account_afspraak_id', 'afspraak_betreft_account_thema', 'afspraak_betreft_account_subthema', 'afspraak_betreft_account_onderwerp', 'afspraak_betreft_account_betreft_id', 'afspraak_betreft_account_eindtijd', 'afspraak_betreft_account_keyphrases']


In [876]:
df_afspraak_betreft_contact = pd.read_csv('../data_clean/Afspraak_betreft_contact_cleaned_fixed.csv', sep=',')
print(df_afspraak_betreft_contact.shape)
print(list(df_afspraak_betreft_contact.columns))
df_afspraak_betreft_contact.drop(['afspraak_betreft_contactfiche_betreft_id'], axis=1, inplace=True)

(2552, 7)
['afspraak_betreft_contactfiche_afspraak_id', 'afspraak_betreft_contactfiche_thema', 'afspraak_betreft_contactfiche_subthema', 'afspraak_betreft_contactfiche_onderwerp', 'afspraak_betreft_contactfiche_betreft_id', 'afspraak_betreft_contactfiche_eindtijd', 'afspraak_betreft_contactfiche_keyphrases']


##### Kolomnamen hernoemen en afspraakt_betreft toevoegen (betreft account = 1, betreft contact = 0)

In [877]:
df_afspraak_acc_gelinkt.columns = ['afspraak_afspraak_id', 'afspraak_thema', 'afspraak_subthema', 'afspraak_onderwerp', 'afspraak_eindtijd', 'afspraak_keyphrases']
df_afspraak_betreft_acc.columns = ['afspraak_afspraak_id', 'afspraak_thema', 'afspraak_subthema', 'afspraak_onderwerp', 'afspraak_eindtijd', 'afspraak_keyphrases']
df_afspraak_betreft_contact.columns = ['afspraak_afspraak_id', 'afspraak_thema', 'afspraak_subthema', 'afspraak_onderwerp', 'afspraak_eindtijd', 'afspraak_keyphrases']

df_afspraak_betreft_acc["afspraak_betreft"] = 1
df_afspraak_acc_gelinkt["afspraak_betreft"] = 1
df_afspraak_betreft_contact["afspraak_betreft"] = 0

In [878]:
acc_gelinkt_id_list = df_afspraak_acc_gelinkt['afspraak_afspraak_id'].unique()
acc_gelinkt_id_list.shape

(2934,)

##### Afspraken mergen en al cleanen

In [879]:
df_afspraken = pd.concat([df_afspraak_betreft_acc, df_afspraak_acc_gelinkt, df_afspraak_betreft_contact], ignore_index=True)
df_afspraken.drop_duplicates(inplace=True)
print(df_afspraken.shape)
print(list(df_afspraken.columns))

(9685, 7)
['afspraak_afspraak_id', 'afspraak_thema', 'afspraak_subthema', 'afspraak_onderwerp', 'afspraak_eindtijd', 'afspraak_keyphrases', 'afspraak_betreft']


In [880]:
df_afspraak_alle = pd.read_csv('../data_clean/Afspraak_alle_fixed.csv', sep=',')
print(df_afspraak_alle.shape)
print(list(df_afspraak_alle.columns))
df_afspraak_alle['afspraak_alle_afspraak_id'].nunique()

(8265, 1)
['afspraak_alle_afspraak_id']


8265

In [881]:
df_afspraken = df_afspraken.merge(df_afspraak_alle, left_on='afspraak_afspraak_id', right_on='afspraak_alle_afspraak_id', how='inner')

df_afspraken['afspraak_thema'] = df_afspraken['afspraak_subthema'].str.replace('\(', '', regex=True)
df_afspraken['afspraak_thema'] = df_afspraken['afspraak_thema'].str.replace('\)', '', regex=True)

df_afspraken['afspraak_onderwerp'] = df_afspraken['afspraak_onderwerp'].str.lower().str.capitalize()
df_afspraken['afspraak_onderwerp'] = df_afspraken['afspraak_onderwerp'].str.replace(' ', '-')
df_afspraken['afspraak_onderwerp'] = df_afspraken['afspraak_onderwerp'].astype(str)

df_afspraken.drop(['afspraak_alle_afspraak_id', 'afspraak_eindtijd', 'afspraak_subthema'], axis=1, inplace=True)

df_afspraken.drop_duplicates(inplace=True)

print(df_afspraken.shape)
print(df_afspraken.columns)

(9678, 5)
Index(['afspraak_afspraak_id', 'afspraak_thema', 'afspraak_onderwerp',
       'afspraak_keyphrases', 'afspraak_betreft'],
      dtype='object')


In [882]:
df_afspraken['afspraak_afspraak_id'].nunique()

7624

##### Waar afspraak_betreft 0 en 1 is maken we er 2 van

In [883]:
# show rows where afspraak_afspraak_id is not unique
list_to_change_afspraak_betreft_to_2 = df_afspraken[df_afspraken['afspraak_afspraak_id'].duplicated(keep=False)]['afspraak_afspraak_id'].unique()

# for every afspraak_afspraak_id that is not unique, change afspraak_betreft to 2
for afspraak_id in list_to_change_afspraak_betreft_to_2:
    df_afspraken.loc[df_afspraken['afspraak_afspraak_id'] == afspraak_id, 'afspraak_betreft'] = 2

df_afspraken.drop_duplicates(inplace=True)
df_afspraken.shape

(7624, 5)

##### Acc_gelinkt = 1 anders 0 -> adhv acc_gelinkt_id_list

In [884]:
for afspraak_id in acc_gelinkt_id_list:
    df_afspraken.loc[df_afspraken['afspraak_afspraak_id'] == afspraak_id, 'afspraak_account_gelinkt'] = 1

df_afspraken['afspraak_account_gelinkt'].fillna(0, inplace=True)

In [885]:
df_afspraken.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7624 entries, 0 to 9684
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   afspraak_afspraak_id      7624 non-null   object 
 1   afspraak_thema            7624 non-null   object 
 2   afspraak_onderwerp        7624 non-null   object 
 3   afspraak_keyphrases       7624 non-null   object 
 4   afspraak_betreft          7624 non-null   int64  
 5   afspraak_account_gelinkt  7624 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 416.9+ KB


##### Nu nog mergen met activiteit_vereist_contact

In [886]:
df_activiteit_vereist_contact = pd.read_csv('../data_clean/Activiteit_vereist_contact_fixed.csv', sep=',')
print(df_activiteit_vereist_contact.shape)
print(list(df_activiteit_vereist_contact.columns))
df_activiteit_vereist_contact['activiteitvereistcontact_reqattendee'].nunique()

(4635, 2)
['activiteitvereistcontact_activityid_id', 'activiteitvereistcontact_reqattendee']


3042

In [887]:
df_afspraken_total = df_afspraken.merge(df_activiteit_vereist_contact, left_on='afspraak_afspraak_id', right_on='activiteitvereistcontact_activityid_id', how='inner')
df_afspraken_total.drop(['activiteitvereistcontact_activityid_id', 'afspraak_afspraak_id'], axis=1, inplace=True)
df_afspraken_total.shape

(4545, 6)

In [888]:
df_afspraken_total.isna().sum()

afspraak_thema                          0
afspraak_onderwerp                      0
afspraak_keyphrases                     0
afspraak_betreft                        0
afspraak_account_gelinkt                0
activiteitvereistcontact_reqattendee    0
dtype: int64

In [889]:
df_afspraken_total.head()

,afspraak_thema,afspraak_onderwerp,afspraak_keyphrases,afspraak_betreft,afspraak_account_gelinkt,activiteitvereistcontact_reqattendee
0,Retentie Lidmaatschap,Ov-ledenbezoek-2020,"china en india , india en china , verschillend...",1,0.0,8E214B8E-F6B8-E911-8104-001DD8B72B62
1,Retentie Lidmaatschap,Ov-klantenbezoek-2018,"zoek , schap",1,0.0,C3B119F3-8502-E911-80F9-001DD8B72B61
2,Retentie Lidmaatschap,Ov-ledenbezoek-2020,"dominique de waele , essentiële sector , gedee...",1,0.0,0CC2FFFC-C56A-E111-B43A-00505680000A
3,Retentie Lidmaatschap,Ov-ledenbezoek-2020,"andere thema ’s , jan dumont , innovatief bedr...",1,0.0,316D6097-E3A8-E711-80EC-001DD8B72B62
4,Retentie Lidmaatschap,Ov-telefoon-2020,"verschillende industriën , verschillende werkg...",1,0.0,1DCF0AC3-D273-E111-B43A-00505680000A


In [890]:
df_afspraken_total.shape

(4545, 6)

Alle niet benodigde dataframes verwijderen

In [891]:
try:
    del df_afspraak_acc_gelinkt
    del df_afspraak_betreft_acc
    del df_afspraak_betreft_contact
    del df_afspraak_alle
    del df_afspraken
    del df_activiteit_vereist_contact
except:
    print('already deleted or non-existent')

### Account en Afspraken mergen

In [904]:
acc_afs_merged = df_account_contact_finance.merge(df_afspraken_total, left_on=['contact_contactpersoon_id'], right_on=['activiteitvereistcontact_reqattendee'], how='left')
acc_afs_merged.drop(['activiteitvereistcontact_reqattendee', 'account_account_id'], axis=1, inplace=True)
acc_afs_merged.shape

(68618, 13)

NaN opvullen

In [905]:
acc_afs_merged['financieledata_boekjaar_range'].fillna(0, inplace=True)
acc_afs_merged['afspraak_betreft'].fillna(-1, inplace=True)
acc_afs_merged['afspraak_account_gelinkt'].fillna(-1, inplace=True)

In [906]:
acc_afs_merged['activiteitscode_naam'].fillna('unknown', inplace=True)
acc_afs_merged['afspraak_thema'].fillna('unknown', inplace=True)
acc_afs_merged['afspraak_onderwerp'].fillna('unknown', inplace=True)
acc_afs_merged['afspraak_keyphrases'].fillna('unknown', inplace=True)

Alle string columns van Afspraak samenvoegen

In [907]:
# combine 'afspraak_thema', 'afspraak_onderwerp', 'afspraak_keyphrases' into one column
acc_afs_merged['afspraak_thema'] = acc_afs_merged['afspraak_thema'].astype(str)
acc_afs_merged['afspraak_onderwerp'] = acc_afs_merged['afspraak_onderwerp'].astype(str)
acc_afs_merged['afspraak_keyphrases'] = acc_afs_merged['afspraak_keyphrases'].astype(str)

replacement_dict = {'\(': '', '\)': '', '\'': '', '\"': '', '\[': '', '\]': ''}

acc_afs_merged['afspraak_keyphrases'] = acc_afs_merged['afspraak_keyphrases'].replace(replacement_dict, regex=True)

In [908]:
acc_afs_merged['afspraak_keyphrases'] = acc_afs_merged['afspraak_thema'] + ', ' + acc_afs_merged['afspraak_onderwerp'] + ', ' + acc_afs_merged['afspraak_keyphrases']
acc_afs_merged['afspraak_keyphrases'] = acc_afs_merged['afspraak_keyphrases'].str.replace('unknown, ', '')
acc_afs_merged['afspraak_keyphrases'] = acc_afs_merged['afspraak_keyphrases'].str.replace(', unknown', '')
acc_afs_merged.drop(['afspraak_thema', 'afspraak_onderwerp'], axis=1, inplace=True)

In [909]:
acc_afs_merged.drop_duplicates(inplace=True)
acc_afs_merged.shape

(68611, 11)

Alle string columns van Account en contact samenvoegen

In [910]:
acc_afs_merged['account_onderneming'] = acc_afs_merged['account_onderneming'].astype(str)
acc_afs_merged['account_adres'] = acc_afs_merged['account_adres'].astype(str)
acc_afs_merged['activiteitscode_naam'] = acc_afs_merged['activiteitscode_naam'].astype(str)
acc_afs_merged['contact_functietitel'] = acc_afs_merged['contact_functietitel'].astype(str)
acc_afs_merged['functie_naam'] = acc_afs_merged['functie_naam'].astype(str)
acc_afs_merged['financieledata_toegevoegde_waarde'] = acc_afs_merged['financieledata_toegevoegde_waarde'].astype(str)
acc_afs_merged['financieledata_boekjaar_range'] = acc_afs_merged['financieledata_boekjaar_range'].astype(str)

acc_afs_merged['account_keyphrases'] = acc_afs_merged['account_onderneming'] + ', ' + acc_afs_merged['account_adres'] + ', ' \
            + acc_afs_merged['activiteitscode_naam'] + ', ' + acc_afs_merged['contact_functietitel'] + ', ' \
            + acc_afs_merged['functie_naam'] + ', financiel toegevoegde waarde ' + acc_afs_merged['financieledata_toegevoegde_waarde'] + ' over ' \
            + acc_afs_merged['financieledata_boekjaar_range'] + ' jaar'

acc_afs_merged['account_keyphrases'] = acc_afs_merged['account_keyphrases'].str.replace('unknown, ', '')
acc_afs_merged['account_keyphrases'] = acc_afs_merged['account_keyphrases'].str.replace(', unknown', '')

acc_afs_merged.drop(['account_onderneming', 'account_adres', 'activiteitscode_naam', 'contact_functietitel', 'functie_naam'
                     , 'financieledata_toegevoegde_waarde', 'financieledata_boekjaar_range'], axis=1, inplace=True)

In [913]:
acc_afs_merged.head()

,contact_contactpersoon_id,afspraak_keyphrases,afspraak_betreft,afspraak_account_gelinkt,account_keyphrases
0,0A9D6946-EB72-E111-B43A-00505680000A,unknown,-1.0,-1.0,"Bedrijf, diensten, overige industrie & dienste..."
1,986BE2D1-B113-EB11-8114-001DD8B72B61,unknown,-1.0,-1.0,"Bedrijf, ,, Bazel 9150 Waasland Oost-vlaandere..."
2,D9303EA2-57E5-EB11-8121-001DD8B72B61,unknown,-1.0,-1.0,"Familiebedrijf, diensten, vastgoed, Melle 9090..."
3,FF015050-8D55-EB11-8117-001DD8B72B61,unknown,-1.0,-1.0,"Familiebedrijf, diensten, vastgoed, Melle 9090..."
4,FF0A3AF6-0D42-E611-80D6-005056B06EC4,unknown,-1.0,-1.0,"Bedrijf, diensten, bouw, Maarke-kerkem 9680 Ou..."


In [914]:
acc_afs_merged.shape

(68611, 5)

Alle niet benodigde dataframes verwijderen

In [915]:
try:
    del df_account_contact_finance
    del df_afspraken_total
except:
    print('already deleted or non-existent')

### Campagne cleanen en mergen

In [918]:
df_campagne = pd.read_csv('../data_clean/Campagne_fixed.csv', sep=',')
print(df_campagne.shape)
print(df_campagne.columns)

(4092, 11)
Index(['campagne_campagne_id', 'campagne_campagne_nr', 'campagne_einddatum',
       'campagne_naam', 'campagne_naam_in_email', 'campagne_reden_van_status',
       'campagne_startdatum', 'campagne_status', 'campagne_type_campagne',
       'campagne_url_voka_be', 'campagne_soort_campagne'],
      dtype='object')
